In [ ]:
#FCNN
def build():
    input_dim = 256 # maximum integer 
    padding_char = 256
    embedding_size = 8 

    inp = Input( shape=(None,), dtype='int64')
    emb = Embedding(input_dim, embedding_size)(inp)
    
    dense0 = Dense(128, activation='relu')(emb)
    
    reduce = tf.reduce_mean(dense0, axis=1)
    
    dense = Dense(128, activation='relu')(reduce)
    out_anomaly = Dense(1, activation='sigmoid', name='anomaly')(dense)
    out_misuse = Dense(total_attacks, activation='softmax', name='misuse')(dense)
 
    model = tf.keras.Model(inp, (out_anomaly, out_misuse))
    model.compile(
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], 
        optimizer='adam',
        metrics=[['binary_accuracy', 'AUC', 'Precision', 'Recall'], ['SparseCategoricalAccuracy']]
    )
    return model

model = build()

model.summary()

In [ ]:
#RNN
def build():
    input_dim = 256 # maximum integer 
    padding_char = 256
    embedding_size = 8 

    inp = Input( shape=(None,), dtype='int64')
    emb = Embedding(input_dim, embedding_size)(inp)
    
    rnn = SimpleRNN(128, return_sequences=True)(emb)
    
    # squish into (None, 64) from (None, None, 64)
    reduce = tf.reduce_mean(rnn, axis=1)
    
    dense = Dense(128, activation='relu')(reduce)
    out_anomaly = Dense(1, activation='sigmoid', name='anomaly')(dense)
    out_misuse = Dense(total_attacks, activation='softmax', name='misuse')(dense)
 
    model = tf.keras.Model(inp, (out_anomaly, out_misuse))
    model.compile(
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], 
        optimizer='adam',
        metrics=[['binary_accuracy', 'AUC', 'Precision', 'Recall'], ['SparseCategoricalAccuracy']]
    )
    return model

model = build()

model.summary()

In [ ]:
#CNN
def build():
    input_dim = 256 # maximum integer 
    padding_char = 256
    embedding_size = 8 

    inp = Input( shape=(None,), dtype='int64')
    emb = Embedding(input_dim, embedding_size)(inp)
    
    cnn = Conv1D( filters=128, kernel_size=3, strides=1, use_bias=True, activation='relu', padding='valid' )(emb)
    
    # squish into (None, 64) from (None, None, 64)
    reduce = tf.reduce_mean(cnn, axis=1)
    
    dense = Dense(128, activation='relu')(reduce)
    out_anomaly = Dense(1, activation='sigmoid', name='anomaly')(dense)
    out_misuse = Dense(total_attacks, activation='softmax', name='misuse')(dense)
 
    model = tf.keras.Model(inp, (out_anomaly, out_misuse))
    model.compile(
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], 
        optimizer='adam',
        metrics=[['binary_accuracy', 'AUC', 'Precision', 'Recall'], ['SparseCategoricalAccuracy']]
    )
    return model

model = build()

model.summary()

In [ ]:
# leap-LSTM custom code
def sample_gumbel(shape, eps=1e-20): 
    """Sample from Gumbel(0, 1)"""
    U = tf.random.uniform(shape,minval=0,maxval=1)
    return -tf.math.log(-tf.math.log(U + eps) + eps)
    
    
def gumbel_softmax_sample(logits, temperature=1e-5): 
    """ Draw a sample from the Gumbel-Softmax distribution"""
    y = logits + sample_gumbel(tf.shape(logits))
    return tf.nn.softmax(y / temperature)


'''
Modern Tensorflow Implementation of leap-LSTM: 
'''


class leapLSTM(keras.layers.Layer):
    '''
    num_cells: number of nodes in the layer
    name: name for the layer
    '''

    def __init__(self, num_cells=100, name='lstm', small_cell_size=10, **kwargs):
        super(leapLSTM, self).__init__(name=name, **kwargs)
        self.num_cells = num_cells
        self.rnn_cell = LSTMCell(self.num_cells)
        self.dense0 = Dense(100, activation="relu", use_bias=True)
        self.dense1 = Dense(2, use_bias=True)

        self.conv1=Conv1D(60, 3, padding = 'same')
        self.conv2=Conv1D(60, 4, padding = 'same')
        self.conv3=Conv1D(60, 5, padding = 'same')
        self.rnn_rev=LSTM(10, return_sequences=True)  # p


    # Build the basic cells. Done automatically for dense layer
    def build(self, input_shape):
        if isinstance(input_shape, list):
            input_shape=input_shape[0]
        if not self.rnn_cell.built:
            with keras.backend.name_scope(self.rnn_cell.name):
                self.rnn_cell.build(input_shape)
                self.rnn_cell.built=True

        self.built=True

    # Used when this layer is part of a network
    # o_inputs is of shape [batch, window size, embedding_dim]
    def call(self, o_inputs, **kwargs):
        # o_inputs is of shape [batch, window size, embedding_dim]
        batch_size=tf.shape(o_inputs)[0]
        win_size=tf.shape(o_inputs)[1]
        enb_size=tf.shape(o_inputs)[2]

        results=tf.TensorArray(dtype = tf.float32, size = win_size)


        conved=tf.concat([
            self.conv1(o_inputs),
            self.conv2(o_inputs),
            self.conv3(o_inputs)],
            axis = -1)
        # shape = [batch, time, 60*3]


        rev_lstm=self.rnn_rev(
            tf.reverse(
                o_inputs, axis=[1]
            )
        )
        # shape = [batch, time, 10]

        f_all=tf.concat([conved, rev_lstm], axis = -1)
        # shape = [batch, time, 190]

        h_end=tf.zeros(shape = [batch_size, 190])
        # shape = [batch, 190]
        
        
        f_all=tf.concat([
            f_all, 
            tf.expand_dims(h_end, axis=1)
        ], axis=1)
        # shape = [batch, time+1, 190]

        '''
        This function is applied to each timestamp of the sequence for the node
        Inputs/Outputs:
            t: current timestamp
            state:  array containing: [ht_1, ct_1]: hidden and candidates from previous timestamp
            res: stacking array containing node output values
        '''
        def _step(t, ht_1, res):

            x_t=o_inputs[:, t, :]

            ff=f_all[:, t+1, :]
   
            
            pi_t = self.dense1(
                self.dense0(
                    tf.concat([ff, x_t], axis=-1)
                )
            )
            
            d_t = gumbel_softmax_sample(pi_t) # tf tensor
            # [None, 2] 2: (keep, skip)
            
            
            # generate output for current timestamp t (Eq 3)
            (out, ht_candidate) = self.rnn_cell(x_t, ht_1)
            
            
            
            # make d_t rank 3 tensor so after slicing it will become rank-2
            d_t = tf.expand_dims(d_t, [1])
            # [None, 1, 2]
            
            ht = [
                d_t[:,:,0] * ht_candidate[0] + d_t[:,:,1] * ht_1[0], # combing keep & skip for lstm state h (element 0)
                d_t[:,:,0] * ht_candidate[1] + d_t[:,:,1] * ht_1[1], # combing keep & skip for lstm state c (element 1)
            ]
            
            # ht = tf.equal(d_t, 0) * ht_candidate + (1 - tf.equal(d_t, 0)) * ht_1
            
            # write node output to index t of array res (returned the updated res array)
            res_updated = res.write(t, out)
            
            return t+1, ht, res_updated
        
        # inital state
        state0 = _generate_zero_filled_state_for_cell(self.rnn_cell, o_inputs, None, None)
        
        *_, final_res = tf.while_loop(
            lambda t, *_: t < win_size, 
            _step,
            (0, state0, results)
        )
        
        
        final_res = final_res.stack()
        # [time, batch, cell_dim]
        final_res = tf.transpose(final_res, [1, 0, 2])
        # [batch, time, cell_dim]
        return final_res


In [ ]:
# leap-LSTM
def build():
    input_dim = 256 # maximum integer 
    padding_char = 256
    embedding_size = 8 

    inp = Input( shape=(None,), dtype='int64')
    emb = Embedding(input_dim, embedding_size)(inp)
    
    rec1 = leapLSTM(128, name="leap")(emb)
    
    # squish into (None, 64) from (None, None, 64)
    reduce = tf.reduce_mean(rec1, axis=1)
    
    dense = Dense(128, activation='relu')(reduce)
    out_anomaly = Dense(1, activation='sigmoid', name='anomaly')(dense)
    out_misuse = Dense(total_attacks, activation='softmax', name='misuse')(dense)
 
    model = tf.keras.Model(inp, (out_anomaly, out_misuse))
    model.compile(
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], 
        optimizer='adam',
        metrics=[['binary_accuracy', 'AUC', 'Precision', 'Recall'], ['SparseCategoricalAccuracy']]
    )
    return model

model = build()

model.summary()

In [ ]:
# BI-LSTM
def build():
    input_dim = 256 # maximum integer 
    padding_char = 256
    embedding_size = 8 

    inp = Input( shape=(None,), dtype='int64')
    emb = Embedding(input_dim, embedding_size)(inp)
    
    lstm1 = Bidirectional(LSTM(32, return_sequences=True))(emb) #[None,40,8]
    lstm2 = Bidirectional(LSTM(128))(lstm1) #[None,32]
    
    
    dense = Dense(128, activation='relu')(lstm2)
    out_anomaly = Dense(1, activation='sigmoid', name='anomaly')(dense)
    out_misuse = Dense(total_attacks, activation='softmax', name='misuse')(dense)
 
    model = tf.keras.Model(inp, (out_anomaly, out_misuse))
    model.compile(
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], 
        optimizer='adam',
        metrics=[['binary_accuracy', 'AUC', 'Precision', 'Recall'], ['SparseCategoricalAccuracy']]
    )
    return model

model = build()

model.summary()

In [ ]:
# LSTM
def build():
    input_dim = 256 # maximum integer 
    padding_char = 256
    embedding_size = 8 

    inp = Input( shape=(None,), dtype='int64')
    emb = Embedding(input_dim, embedding_size)(inp)
    
    rec1 = leapLSTM(128, name="leap")(emb)
    
    # squish into (None, 64) from (None, None, 64)
    reduce = tf.reduce_mean(rec1, axis=1)
    
    dense = Dense(128, activation='relu')(reduce)
    out_anomaly = Dense(1, activation='sigmoid', name='anomaly')(dense)
    out_misuse = Dense(total_attacks, activation='softmax', name='misuse')(dense)
 
    model = tf.keras.Model(inp, (out_anomaly, out_misuse))
    model.compile(
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], 
        optimizer='adam',
        metrics=[['binary_accuracy', 'AUC', 'Precision', 'Recall'], ['SparseCategoricalAccuracy']]
    )
    return model

model = build()

model.summary()

In [ ]:
# GCNN
def build():
    input_dim = 256 # maximum integer 
    padding_char = 256
    embedding_size = 8 

    inp = Input( shape=(None,), dtype='int64')
    emb = Embedding( input_dim, embedding_size)(inp)
    filt = Conv1D( filters=128, kernel_size=3, strides=1, use_bias=True, activation='relu', padding='valid' )(emb)
    attn = Conv1D( filters=128, kernel_size=3, strides=1, use_bias=True, activation='sigmoid', padding='valid')(emb)
    gated = Multiply()([filt,attn])
    feat = GlobalMaxPooling1D()( gated )
    dense = Dense(128, activation='relu')(feat)
    out_anomaly = Dense(1, activation='sigmoid', name='anomaly')(dense)
    out_misuse = Dense(11, activation='softmax', name='misuse')(dense)
 
    model = tf.keras.Model(inp, (out_anomaly, out_misuse))
    model.compile(
        loss=['binary_crossentropy', 'sparse_categorical_crossentropy'], 
        optimizer='adam',
        metrics=[['binary_accuracy', 'AUC', 'Precision', 'Recall'], ['SparseCategoricalAccuracy']]
    )
    return model

model = build()

model.summary()